<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/Making_local_docs_with_sent_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq sentence-transformers pdfminer.six spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
from pdfminer.high_level import extract_text
from sentence_transformers import util
import numpy as np
from spacy.lang.en import English
import spacy, os,sqlite3, pdfminer
from sentence_transformers import SentenceTransformer

nlp = spacy.load('en_core_web_sm')

In [ ]:
# conn = sqlite3.connect('Knowledge-database-29-3-24.db')
# cursor=conn.cursor()
# cursor.execute("DROP TABLE documents")
# conn.commit()
# conn.close()

In [ ]:
# Establish a connection to the SQLite database
conn = sqlite3.connect('Knowledge-database-29-3-24.db')

# Create a cursor object
cursor = conn.cursor()

# Execute SQL queries, change embedding type to somethiiing more suitbale in future
# In final version normalise this by having a documents table, not need to have title here
cursor.execute('''CREATE TABLE IF NOT EXISTS documents (
                    id INTEGER PRIMARY KEY,
                    document_ID INTEGER,
                    document_title TEXT,
                    sentence TEXT,
                    embedding BLOB)''')

# Commit the transaction
conn.commit()
conn.close()


In [ ]:
def read_pdf(file):
    text = extract_text(file)
    return text

In [ ]:
DIR = "/content/drive/MyDrive/WRAP food reports"

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
  # Establish a connection to the SQLite database
conn = sqlite3.connect('Knowledge-database-29-3-24.db')

# Create a cursor object
cursor = conn.cursor()

for num, title in enumerate(os.listdir(DIR)):

    path = DIR + "/" + title
    # read doc
    text = read_pdf(path)
    text = text.replace("\n", "")
    # split into sentences
    doc = nlp(text)
    # loop to embed sentences
    for sent in doc.sents:
        embedding = model.encode(sent.text)
        cursor.execute("""INSERT INTO documents (document_id,
                                                document_title,
                                                sentence,
                                                embedding) VALUES (?, ?, ?, ?)""",
                                                 (num, title, sent.text, embedding))

conn.commit()
conn.close()

In [ ]:

def find_top_k_relevance(question=["No question"], model=None, n=5, db="Knowledge-database-29-3-24.db"):
  q = model.encode(question)
  scores = {}
  conn = sqlite3.connect(db)

# Create a cursor object
  cursor = conn.cursor()
  # load row by row
  cursor.execute('SELECT * FROM documents')
  for row in cursor:
    scores[row[0]] = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()

  return dict(sorted(scores.items(), key = lambda x: x[1], reverse = True)[:n])
find_top_k_relevance(question =["How do I increase shelf life of lettuce?"], model=model)

{366: array([[0.711703]], dtype=float32),
 365: array([[0.6810641]], dtype=float32),
 364: array([[0.67368263]], dtype=float32),
 367: array([[0.66278386]], dtype=float32),
 299: array([[0.56608444]], dtype=float32)}

In [ ]:
import pprint
conn = sqlite3.connect("Knowledge-database-29-3-24.db")
cursor = conn.cursor()
cursor.execute("""SELECT * FROM documents
                 WHERE id = 366""")
rows = cursor.fetchone()
conn.close()
pprint.pprint(f"{rows[3]} context: {rows[2]}")

('Other effective methods included storing lettuce in a plastic bag or '
 'unwrapped in the crisper drawer. context: WRAP-Evidence Review Plastic '
 'Packaging and Fresh Produce 171218.pdf')


#### I think from here the trick will be to increase the sentence lenght. But first, come up with a standard set of questions and then the relevance of the responses can be graded etc

https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

## Question list (30)
Have the relevant document for each so you can measure, recall, precision etc

Best to complete this after asking the question to the system?

|Doc_ID|Name|Relevant_Q|
|---|---|---|
|1|Evidence & insights - reducing household food waste & plastic packaging (packaged vs loose)|1,2,3,4,5,6,7|
|2|Food waste collection guidance |8|
|3|Food Waste Reduction Roadmap - Executive Summary|9|
|4|Food Waste Reduction Roadmap - Executive Summary|9|
|5|Food Waste Reduction Roadmap - Manufacturers|9|
|6|Food Waste Reduction Roadmap - Primary Producers|9,10|
|7|Food Waste Reduction Roadmap - Retail|9,11|
|8|Identifying impacts from food and farm digestates|12,13|
|9|Industry Guidance - Dealing with Household Food Waste at AD Facilities - Management of Liners|14,8|
|10|Literature review - relationship between household food waste collection and food waste prevention||
|11|The food waste reduction roadmap progress report 2022||
|12|Towards the 2030 Food Waste Commitment -- setting our coalition baseline||
|13|Upscaling farm food waste measurement in the UK||
|14|WRAP-Evidence Review Plastic Packaging and Fresh Produce 171218||

1. "How should I store bananas?"
2. "What temperature should I keep milk?"
3. "How do I increase shelf life of lettuce?
4. "How does selling fresh produce loose, affect waste?"
5. "What is a best-before date for"?
6. "What is a use-by date for"?
7. "How does removing date labels help?"
8. "What are the reasons to not participate in a food waste scheme?"
9. "What is the UN's sustainable development goal with respect to food?"
10. "Which document pertains to primary producers and food waste?"
11. "Which document pertains to retailers and food waste?"
12. "What is the most common use for digestate?"
13. "What affects the costs of valorisation?"
14. "How many tonnes of food waste are there each year?
15. "What is the benefit of caddy liners?"
16. "How to prevent food loss?"
17. "How to prevent food waste?"
18. "How does composting affect food waste?"
19. "What are the main drivers of food waste?"
20. "Online shopping and food waste"
21. "What is involved with Courthold 2030?"
